You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Introduction:
- Brief overview of the current state of artificial intelligence
- Importance of staying updated on the latest trends in AI

Key Points:
1. Latest Trends in Artificial Intelligence
   - Deep learning and neural networks
   - Natural language processing
   - Computer vision
   - Autonomous vehicles
   - AI ethics and bias

2. Key Players in the AI Industry
   - Google DeepMind
   - IBM Watson
   - Open

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction

Artificial intelligence (AI) has rapidly become a transformative force in various industries, revolutionizing the way we work, interact, and live. Staying informed about the latest trends and advancements in AI is crucial for professionals in the tech industry, AI enthusiasts, and business owners looking to adopt AI technology. In this blog post, we will explore the current state of AI, delve into the latest trends shaping the industry, highlight key players in the AI landscape, and discuss noteworthy news in artificial intelligence.

## Latest Trends in Artificial Intelligence

### Deep Learning and Neural Networks
Deep learning and neural networks are at the forefront of AI innovation, enabling machines to learn from vast amounts of data and make complex decisions. This technology powers many AI applications, from virtual assistants

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction

Artificial intelligence (AI) has rapidly become a transformative force in various industries, revolutionizing the way we work, interact, and live. Staying informed about the latest trends and advancements in AI is crucial for professionals in the tech industry, AI enthusiasts, and business owners looking to adopt AI technology. In this blog post, we will explore the current state of AI, delve into the latest trends shaping the industry, highlight key players in the AI landscape, and discuss noteworthy news in artificial intelligence.

## Latest Trends in Artificial Intelligence

### Deep Learning and Neural Networks
Deep learning and neural networks are at the forefront of AI innovation, enabling machines to learn from vast amounts of data and make complex decisions. This technology powers many AI applications, from virtual assistants to self-driving cars, and continues to advance rapidly.

### Natural Language Processing
Natural language processing (NLP) is another key trend in AI, allowing machines to understand and generate human language. NLP is widely used in chatbots, language translation services, and sentiment analysis tools, making human-machine interactions more seamless and intuitive.

### Computer Vision
Computer vision is the field of AI that enables machines to interpret and understand visual information from the world around them. Applications of computer vision range from facial recognition systems to autonomous drones, with potential implications for various industries.

### Autonomous Vehicles
The development of autonomous vehicles represents a significant advancement in AI technology, with companies like Tesla and Waymo leading the way in self-driving car technology. AI algorithms power these vehicles, enabling them to navigate roads, interpret traffic signals, and make split-second decisions to ensure passenger safety.

### AI Ethics and Bias
As AI becomes more integrated into society, the importance of ethical considerations and addressing bias in AI systems has come to the forefront. Ensuring that AI technologies are developed and deployed responsibly is crucial to mitigating potential risks and ensuring fair and equitable outcomes for all.

## Key Players in the AI Industry

### Google DeepMind
Google DeepMind is known for its cutting-edge research in AI and its development of AlphaGo, the AI system that defeated human champions in the game of Go. DeepMind continues to push the boundaries of AI with its research in reinforcement learning and neural network architectures.

### IBM Watson
IBM Watson is a pioneer in the field of AI, known for its cognitive computing capabilities and applications in healthcare, finance, and other industries. Watson's natural language processing and machine learning capabilities have been instrumental in driving AI innovation.

### OpenAI
OpenAI is a research organization dedicated to advancing AI in a safe and beneficial manner. Known for its work on reinforcement learning, language models, and robotics, OpenAI aims to democratize AI technology and ensure its responsible development.

### Microsoft AI
Microsoft AI encompasses a range of products and services that leverage AI technology, including Azure AI, Cognitive Services, and conversational AI tools. Microsoft's AI initiatives focus on empowering businesses to harness the power of AI for improved productivity and efficiency.

### Amazon AI
Amazon AI powers a wide range of Amazon products and services, from personalized recommendations to voice recognition technology. Amazon's AI capabilities drive customer experiences across e-commerce, cloud computing, and smart home devices, showcasing the potential of AI in enhancing everyday interactions.

## Noteworthy News in Artificial Intelligence

### Breakthroughs in AI Research
Recent breakthroughs in AI research have led to advancements in reinforcement learning, generative models, and explainable AI. These developments have the potential to reshape industries and drive innovation in AI applications.

### AI Applications in Various Industries
AI is being increasingly integrated into various industries, from healthcare and finance to retail and manufacturing. Applications of AI technology include predictive analytics, personalized recommendations, and process automation, transforming how businesses operate and deliver value to customers.

### Impact of AI on Job Market and Society
The rise of AI technology has raised concerns about its impact on the job market and society at large. While AI has the potential to create new opportunities and drive economic growth, it also poses challenges related to job displacement, workforce reskilling, and ethical considerations surrounding AI deployment.

## Conclusion and Call to Action

As artificial intelligence continues to evolve and shape the future of technology, staying informed about the latest trends, key players, and noteworthy news in AI is essential for professionals and businesses alike. By keeping up with the latest advancements in AI and understanding its potential impact on different sectors, individuals can make informed decisions about adopting AI technology and navigating the opportunities and challenges it presents.

In conclusion, the future of artificial intelligence holds immense promise for innovation and transformation across industries. By embracing AI technology responsibly and ethically, we can harness its potential to drive progress, improve decision-making, and enhance human experiences in a rapidly evolving digital landscape.

Remember, the future of artificial intelligence is in our hands, and staying informed and engaged with the latest developments in AI is key to shaping a future that benefits us all.

*Stay tuned for more updates on the latest trends, key players, and noteworthy news in the dynamic world of artificial intelligence.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "Christiano Ronaldo"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Christiano Ronaldo.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Rise of Christiano Ronaldo: A Detailed Look at the Soccer Superstar

Introduction: 
- Brief overview of Christiano Ronaldo's career and achievements
- Teaser of what the article will cover

Key Points:
1. Latest Trends:
- Recent transfer to Manchester United
- Impact on team performance and fan engagement
- Social media presence and influence

2. Key Players:
- Comparison with other soccer superstars like Lionel Messi
- Collaborations with brands and endorsements
- Contributions to charity and philanth

I now can give a great answer

Final Answer: 
# The Rise of Christiano Ronaldo: A Detailed Look at the Soccer Superstar

## Introduction:

Christiano Ronaldo, the renowned soccer superstar, has had a remarkable career marked by numerous achievements and accolades. From his early days at Sporting Lisbon to his stints at Manchester United, Real Madrid, and Juventus, Ronaldo has consistently showcased his exceptional skills and passion for the game. In this article, we will delve into the latest trends in Christiano Ronaldo's career, compare him with other key players in soccer, provide noteworthy updates on his personal life, and offer insights into his future plans and retirement speculations.

## Latest Trends in Christiano Ronaldo's Career:

Recently, Christiano Ronaldo made a highly anticipated return to Manchester United, creating a buzz among fans and critics alike. His impact on the team's performance has been significant, with his goals and assists boosting the overall morale and

In [16]:
Markdown(result)

# The Rise of Christiano Ronaldo: A Detailed Look at the Soccer Superstar

## Introduction:

Christiano Ronaldo, the renowned soccer superstar, has had a remarkable career marked by numerous achievements and accolades. From his early days at Sporting Lisbon to his stints at Manchester United, Real Madrid, and Juventus, Ronaldo has consistently showcased his exceptional skills and passion for the game. In this article, we will delve into the latest trends in Christiano Ronaldo's career, compare him with other key players in soccer, provide noteworthy updates on his personal life, and offer insights into his future plans and retirement speculations.

## Latest Trends in Christiano Ronaldo's Career:

Recently, Christiano Ronaldo made a highly anticipated return to Manchester United, creating a buzz among fans and critics alike. His impact on the team's performance has been significant, with his goals and assists boosting the overall morale and engagement of both players and supporters. Moreover, Ronaldo's social media presence and influence have continued to grow, with millions of followers eagerly awaiting his latest updates and endorsements.

## Comparison with Key Players in Soccer:

When it comes to comparing Christiano Ronaldo with other soccer superstars like Lionel Messi, the debate rages on. While Messi may have a different playing style and career trajectory, Ronaldo's collaborations with brands and endorsements have set him apart in the world of sports marketing. Additionally, his contributions to charity and philanthropic efforts have endeared him to fans worldwide, showcasing his dedication to making a positive impact beyond the field.

## Noteworthy News and Updates:

In terms of noteworthy news, updates on Christiano Ronaldo's personal life, injuries, and recovery process are always in the spotlight. Fans are eager to learn about his family, training routines, and future career plans, as well as any speculations surrounding his eventual retirement. As Ronaldo continues to push boundaries and break records, the soccer world eagerly awaits his next move and the legacy he will leave behind.

## Conclusion:

In conclusion, Christiano Ronaldo remains a dominant force in the world of soccer, captivating audiences with his talent, charisma, and dedication to the game. As he navigates through the highs and lows of his career, fans can expect nothing but excellence and determination from this iconic athlete. Stay tuned for more updates on Christiano Ronaldo's journey, and continue to support the soccer superstar as he continues to make history on and off the field. Thank you for joining us on this detailed look at the rise of Christiano Ronaldo.

**Call to Action:** Stay informed about the latest updates on Christiano Ronaldo by following his official website, Manchester United's transfer news, and reputable soccer news sources like ESPN and BBC Sport. Engage with Christiano Ronaldo's social media profiles for exclusive content and behind-the-scenes insights into his career and personal life. Join the conversation and show your support for the soccer superstar as he continues to inspire millions of fans worldwide.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).